<a href="https://colab.research.google.com/github/zakarka2006/famcs2024/blob/main/famcs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Установите библиотеки и скачайте картинки, затем по очереди запускайте блоки с кодом.

In [ ]:
!npm install -g degit
!pip install opencv-python pytesseract

In [ ]:
!npx degit zakarka2006/famcs2024/images -f ./images

# Letters 1
Начнем с букв: почему-то я сразу начал думать в сторону opencv, как минимум для решения пунктов 2-3, поэтому решение получилось немного "костыльным". Я вырезал отдельные картинки букв размером 60х60 пикселей, но перед этим выкрутил контраст на максимум, чтобы не терять деталей. (Все картинки в папке `./images`)

Как я считаю буквы:
1. Преобразование картинки: функция переводит картинку в ч/б формат и увеличивает контрастность
2. Предобработка изображения: функция загружает изображение и преобразует его в черно-белое. Затем применяется бинаризация с инверсией, чтобы буквы стали белыми на черном фоне.
3. Поиск контуров: на этом этапе я нахожу контуры на бинарном изображении. Найденные контура - это области, содержащие буквы.
4. Подготовка шаблонов для букв: функция создает шаблоны для каждой буквы. Изображения букв предварительно обрабатываются и извлекаются их контуры. Шаблоны сохраняются в словаре для последующего использования.
5. Сопоставление контуров: каждый найденный контур на картинке сопоставляется с шаблоном буквы, в итоге для каждого контура выбирается более подходящий шаблон буквы

Конечно, я пробовал прикрутить ocr для этой задачи, но не все буквы определялись правильно со 100% вероятностью. Поэтому отошел от этой идеи. Работу этого кода проверял вручную на небольших картинках по 8х8 букв, ошибок не нашел, так что должно работать :D.

In [ ]:
import cv2
import numpy as np
from collections import Counter
from PIL import Image, ImageEnhance

Image.MAX_IMAGE_PIXELS = 100000000
def maximize_contrast(image_path):
    image = Image.open(image_path)
    image = image.convert("L")
    enhancer = ImageEnhance.Contrast(image)
    max_contrast_image = enhancer.enhance(10)
    max_contrast_image.save(f"{image_path[:-4]}_contrast.png")

def preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, binary_image = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    return binary_image

def find_contours(binary_image):
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def create_letter_templates():
    templates = {}
    for letter in "ABCDEFGHIJKLMNOPQRSTUVWXYZ":
        template_path = f"./images/letters/{letter}.png"
        template_image = preprocess_image(template_path)
        contours = find_contours(template_image)
        if contours:
            templates[letter] = contours
    return templates

def match_letters(image, templates):
    detected_letters = []
    contours = find_contours(image)
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        letter_image = image[y:y+h, x:x+w]
        max_match = float("inf")
        matched_letter = ""
        for letter, template_contours in templates.items():
            for template_contour in template_contours:
                match = cv2.matchShapes(template_contour, contour, cv2.CONTOURS_MATCH_I3, 0.0)
                if match < max_match:
                    max_match = match
                    matched_letter = letter
        detected_letters.append(matched_letter)
    return detected_letters

def count_letters():
    maximize_contrast("./images/letters.png")
    binary_image = preprocess_image("./images/letters_contrast.png")
    templates = create_letter_templates()
    detected_letters = match_letters(binary_image, templates)

    letter_count = Counter(detected_letters)
    sorted_letters = sorted(letter_count.items())
    total_count = 0

    for letter, count in sorted_letters:
        if not letter.isalpha():
            continue
        total_count += count
        print(f"{letter}: {count}")

    print(f"Total: {total_count}")

count_letters()

# Letters 2
Самый простой пункт, по моему мнению, т.к. все просто сводится к нахождению контуров букв и отрисовки рамок вокруг них.

Итоговая картинка лежит в `./images/letters_rectangles.png`



In [ ]:
import cv2
import numpy as np
import time

def add_bounding_boxes_letters():
    padding = 2
    thickness = 3
    maximize_contrast("./images/letters.png")
    img = cv2.imread("./images/letters_contrast.png")

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    kernel = np.ones((3, 3), np.uint8)
    img_erode = cv2.erode(thresh, kernel, iterations=1)

    contours, _ = cv2.findContours(img_erode, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    output = cv2.imread("./images/letters.png")
    padding += thickness
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        x_pad = max(x - padding - 1, 0)
        y_pad = max(y - padding - 1, 0)
        dx = (0 - (x - padding - 1)) if x_pad == 0 else 0 # для букв возле левой границы
        w_pad = w + 2*padding + 1 - dx
        h_pad = h + 2*padding + 1

        border_color = (0, 0, 0)
        cv2.line(output, (x_pad, y_pad), (x_pad + w_pad, y_pad), border_color, thickness)
        cv2.line(output, (x_pad, y_pad + h_pad), (x_pad + w_pad, y_pad + h_pad), border_color, thickness)
        cv2.line(output, (x_pad + w_pad, y_pad), (x_pad + w_pad, y_pad + h_pad), border_color, thickness)
        if dx == 0:
            cv2.line(output, (x_pad, y_pad), (x_pad, y_pad + h_pad), border_color, thickness)

    cv2.imwrite("./images/letters_rectangles.png", output)
    print("Done")

add_bounding_boxes_letters()

# Letters 3
Можно немного изменить второй код и вместо рамок рисовать фон под буквами
Как работает:
1. Поиск контуров
2. Выбор самого встречающегося цвета в пределах контура
3. Наложение прямоугольной рамки
4. Наложение буквы на итоговую картинку с помощью маски

Итоговая картинка лежит в `./images/letters_backgrounds.png`


In [34]:
import cv2
import numpy as np
import time

def add_background_letters():
    maximize_contrast("./images/letters.png")
    img = cv2.imread("./images/letters_contrast.png")

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    margin = 3

    imgc = cv2.imread("./images/letters.png")
    output = imgc.copy()
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)

        letter_color = np.mean(output[y:y+h, x:x+w], axis=(0, 1))

        background_color = [255 - int(c) for c in letter_color]

        cv2.rectangle(output, (x-margin, y-margin), (x+w+margin, y+h+margin), background_color, -1)

        letter_region = imgc[y:y+h, x:x+w]
        mask = binary[y:y+h, x:x+w]
        mask_inv = cv2.bitwise_not(mask)
        bg = cv2.bitwise_and(output[y:y+h, x:x+w], output[y:y+h, x:x+w], mask=mask_inv)
        fg = cv2.bitwise_and(letter_region, letter_region, mask=mask)
        output[y:y+h, x:x+w] = cv2.add(bg, fg)

    cv2.imwrite("./images/letters_backgrounds.png", output)
    print("Done")

add_background_letters()